### Análisis de Datos con Pandas
En programación 1, aprendimos a abrir archivos csv usando el paquete `csv`.

Hagamos un repaso 😀

### Paquete CSV

Comencemos leyendo el archivo con csv. Como podrán recordar, extraer información puede ser un poco tedioso ya que para poder obtener todos los valores en una columna, tenemos que recorrer todas las filas del archivo csv.

En este caso vamos a recolectar los precios y los precios clasificados por barrios

In [19]:
import csv
lista_precios = []
precio_por_barrio = {}

with open('listings.csv') as File:
    next(File)
    reader = csv.reader(File, delimiter=',')
    for fila in reader:
        precio = float(fila[9].replace('$', '').replace(',',''))
        lista_precios.append(precio)
        #por barrio
        barrio = fila[5] # neighbourhood
        if barrio in precio_por_barrio:
            precio_por_barrio[barrio].append(precio)
        else:
            precio_por_barrio[barrio] = [precio]

**En** el código de arriba generamos una lista con todos los precios que aparecen en el archivo listing.csv y un diccionario donde las claves son los barrios y los valores son todos los precios de ese barrio. A continuación hacemos una breve exploración de esos datos

In [20]:
import matplotlib.pyplot as plt
plt.hist(lista_precios)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
sum(lista_precios)/len(lista_precios)

In [ ]:
precio_por_barrio.keys()

In [ ]:
# Calculamos el precio promedio en el barrio Bijlmer-Centrum
sum(precio_por_barrio['Bijlmer-Centrum'])/len(precio_por_barrio['Bijlmer-Centrum'])

### Paquete Pandas 💗🐼💗🐼💗🐼💗🐼💗

El paquete pandas es muy utilizado en el análisis de datos para leer y manipular archivos de diferentes tipos, en particular el csv.

* Uno de los beneficios más importantes de usar pandas es que nos permite acceder a los datos como si estos fueran una tabla, es decir, puedo acceder a las filas o columnas sin importar cuál es el archivo base.

* Además, nos permite tener los datos asociados a su índice, algo que por ahora no tiene mucho sentido pero lo tendrá más adelante

* Y los datos se ven muy lindos 💣

Según sus creadores pandas es:

> pandas is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

La documentación de pandas la encuentran acá https://pandas.pydata.org/docs/

In [1]:
import pandas as pd #casi todo el mundo importa pandas así

In [2]:
data = pd.read_csv('listings.csv')

Podemos ver los primeros $n$ registros del archivo utilizando el método `head`

In [3]:
data.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,23726706,Private room 20 minutes from Amsterdam + Break...,122619127,Patricia,NaN,IJburg - Zeeburgereiland,52.34916,4.97879,Private room,88,2,78,2022-05-29,1.53,1,66,11,038469D9AA6BDF1142CE
1,35815036,"Vrijstaand vakantiehuis, privé tuin aan het water",269425139,Lydia,NaN,Noord-Oost,52.42419,4.95689,Entire home/apt,105,3,95,2022-06-02,2.65,1,243,36,NaN


Vamos a ver cómo calculamos el precio promedio de los listings? y por barrio?

In [4]:
data['price'].mean()

198.01960149036125

In [ ]:
data[data['neighbourhood']=='Bijlmer-Centrum']['price'].mean()

### Beneficios de utilizar pandas
Algunos de los beneficios que vemos a simple vista si usamos pandas comparado con el módulo CSV:


### 🐼 Conceptos Básicos 🐼

- Las estructuras de datos más usadas en pandas son los Dataframes y las Series
- Los DataFrames pueden ser pensados como una tabla y las Series como las columnas de la tabla

In [ ]:
#Veamos como es el tipo de datos de DataFrame
type(data)

In [ ]:
# Veamos como es el tipo de datos de Series
type(data['price'])

Podemos utilizar el comando `head` tanto para DataFrame como para Series.

In [ ]:
data.head(2)

In [ ]:
data['price'].head(5)

Además tenemos los siguientes métodos básicos:

* `tail` Nos muestra los _ultimos_ $n$ registros del DataFrame o de la Serie.

In [ ]:
data.tail(2)

In [ ]:
data['price'].tail(5)

In [ ]:
print(data['price'].tail(5))

* `min` y `max` que permiten obtener el valor mínimo y máximo de una serie, respectivamente.

In [ ]:
data['price'].min()

In [ ]:
data['price'].max()

* `describe` que realiza una descripción básica de una Serie.
  * La cantidad de datos no nulos que contiene.
  * El máximo y el mínimo.
  * El promedio de los valores.
  * La desviación estándar de los mismos.
  * Los _cuartiles_: Son los tres valores que dividen a un conjunto de datos en cuatro partes iguales.


In [5]:
data['price'].describe()

count    6173.000000
mean      198.019601
std       140.546979
min         0.000000
25%       115.000000
50%       160.000000
75%       240.000000
max      2500.000000
Name: price, dtype: float64

Podemos obtener mas información sobre `describe` y sobre cualquier otro método, utilizando la función integrada en el interprete `help`.

In [ ]:
help(pd.Series.describe)

Otro método útil para una Serie es `value_counts`. Consultemos la ayuda para ver que hace:

In [ ]:
help(pd.Series.value_counts)

Este método nos da la frecuencia de ocurrencia de cada valor para una serie. Por ejemplo, podemos ver cuantos departamentos hay en alquiler en cada barrio:

In [ ]:
len(precio_por_barrio["Centrum-West"])

In [ ]:
data['neighbourhood'].value_counts()

Para un DataFrame, es importante conocer la _estructura_ del mismo, es decir, cuantas columnas y cuantas filas contiene. Esto lo podemos ver utilizando el atributo `shape`.

In [ ]:
data.shape

Esto nos indica que el archivo contiene 6173 observaciones y 18 columnas o _variables_.

### Concepto de índice
Como probablemente ya vieron en la materia de base de datos, los índices son un elemento muy importantes cuando almacenamos los datos.
Es lo que nos permite referirnos a un dato de forma únivoca. Para determinar el índice de un dataset, vamos a buscar una variable/columna/característica única, es decir, que no tenga repetidos.

Por ejemplo, si en nuestros datos del listing queremos identificar una publicación en particular, ¿como lo haríamos? ¿Con el precio? ¿Con el nombre de la publicación? ¡Probablemente no! Lo que haríamos es referirnos a un listing por su id.

Una característica muy importante de los DFs y Series es que **siempre tienen un índice**. Cuando pandas abre un archivo, automáticamente genera un header, los nombres de las columnas, y genera un índice automáticamente que va de 0 a la longitud del archivo -1.

En nuestros datos, vemos que si bien pandas genera un índice automático, no es el índice que queremos usar, el id del listing. Para poder cambiarlo, vamos a usar el método `df.set_index`


In [ ]:
# podemos obtener el nombre de las cabeceras utilizando el atributo `columns`.
data.columns

In [ ]:
data.set_index('id').head(2)

In [ ]:
data.head()

¿Que paso? Cambiamos el índice pero no se vio reflejado en el objeto `data`... ¿Que puede estar pasando?

## La mutabilidad

En la unidad de Programación Orientada a Objetos, vimos que existían dos tipos de métodos:

* Los **métodos puros** que no modifican el estado del objeto sobre el cuál lo llamamos.
* Los **métodos modificadores** que modifican el estado del objeto sobre el cuál lo llamamos.

Por si están olvidados, veamos un ejemplo con una lista de python. Para ordenar una lista, podemos utilizar el método `.sort` ...

In [ ]:
A = [1, 2, 3, 4, 2, 1, 1]
A.sort()
print(A)

... que modifica la lista "in place". En el mismo objeto A se reordenan los elementos. O bien podemos hacer

In [ ]:
A = [1, 2, 3, 4, 2, 1, 1]
A_ordenada = sorted(A)
print(A_ordenada)

... que _devuelve_ o _retorna_ una nueva lista con los mismos elementos que A, pero en orden. Notar que aca la lista originar A **sigue existiendo**

In [ ]:
print(A)

Si queremos, podemos "pisar" la lista A con los valores ordenados simplemente asignando la misma variable al resultado

In [ ]:
A = [1, 2, 3, 4, 2, 1, 1]
A = sorted(A)
print(A)

En pandas ocurre lo mismo. Casi todos los métodos que modifiquen un dataframe o una serie aceptaran una versión `inplace` pasando el argumento `inplace=True` al método. Si no lo hacemos, pandas se comportará _retornando_ un nuevo objeto con las modificaciones que le pedimos. Luego será nuestra responsabilidad asignar ese objeto a una variable (que puede o no ser la misma, dependiendo de lo que querramos).

In [ ]:
# data.set_index('id', inplace = True) #cambia el índice inplace
data = data.set_index("id")
data.head(2)

**Convención**: Por convención, evitaremos el uso de `inplace=True` y trabajaremos con métodos puros y objetos nuevos siempre que sea posible.

## Indexar
Pandas nos permite acceder a la información dentro de sus DFs y Series mediante sus índices con los atributos .loc y .iloc. `loc` nos permite acceder a los índices por su valor y `iloc` por su posición.
Vamos a poder acceder a un índice o a varios. Y también vamos a poder incluir las columnas

* Primer sabor de loc: Si le pasamos un indice con un valor particular nos devueelve una Serie, donde el indice de la serie son las columnas del dataframe.

In [ ]:
data.loc[31553121]

* Segundo sabor de loc: Podemos pasarle una lista de indices y nos devuelve un dataframe solo con aquellos indices

In [ ]:
data.loc[[23726706, 35815036]]

* Tercer sabor de loc: Podemos pasarle, ademas, que columna nos interesa. En este caso nos devolverá una serie solo con los indices que le pedimos

In [ ]:
data.loc[[31553121], 'price']

* Cuarto sabor de loc: Podemos pasarle una lista de las columnas que queremos. En este caso, nos devolvera un dataframe, que contiene solo los indices que le pedimos y solo las columnas que le pedimos.

In [ ]:
data.loc[[23726706, 35815036], ['price']]

**Convención** Si bien `iloc`, que sirve para acceder a los indices y a las columnas por su posicion y no por su nombre puede ser de utilidad en ciertas situaciones, _lo evitaremos_ siempre que sea posible.

## Haciendo cambios al dataset

### Agregar una columna

Si que

## Ejercicio 1 (15-20 mins):

1. Cargar el archivo `pokemon_data.txt` a un archivo. Deberán investigar el formato del archivo y buscar en las opciones de la función `read_csv` para poder cargarlo correctamente.
2. ¿Como se llaman las columnas de nuestro dataset? ¿Que tipo tiene cada una?
3. Elegir (y setear) un indice para el dataset.
4. Obtener el ataque promedio de un pokemon.
5. Obtener el ataque promedio de un pokemon de primera generacion.
6. Contar cuantos pokemones legendarios y no legendarios hay.
7. Investigar el método `.sort_values` utilizando la función `help` o la documentación de pandas. Utilizarlo para ordenar los pokemons por puntos de salud (HP) desde el mayor al menor.

In [49]:
# Ej 1
df = pd.read_csv('pokemon_data.txt', sep='\t', header=0, engine='python')
df.head(2)


,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False


In [41]:
# Ej 2
df.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [44]:
# Ej 3
df.set_index('#').head(2)

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False


In [45]:
# Ej 4
df['Attack'].mean()

79.00125

In [47]:
# Ej 5
df[df['Generation'] == 1]['Attack'].mean()

76.63855421686748

In [53]:
# Ej 6
df[df['Legendary'] == False]['Name'].count(),df[df['Legendary'] == True]['Name'].count()

(735, 65)

In [63]:
# Ej 7
df['HP'].sort_values(ascending=False)

261    255
121    250
217    190
351    170
655    165
      ... 
139     20
381     20
388     20
55      10
316      1
Name: HP, Length: 800, dtype: int64

### Crear un DataFrame
También podemos crear un DataFrame manualmente. ¡Hay muchas maneras de hacerlo!

Instanciamos el objeto diciendo que indices y columnas queremos que tenga y luego "rellenamos" esa tabla:

In [ ]:
df = pd.DataFrame(index = [1,2,3,4], columns = ['col1', 'col2'])

# Podemos utilizar loc/iloc para "rellenar" el DF!

df.loc[1, "col1"] = 2
df.loc[2, "col2"] = 5
df.loc[2, "col2"] = 3
# print(df.loc[2, "col2"])
print(df)

Sin embargo, lo mas común es leer directamente el DataFrame de una fuente de datos tabular con la familia de funciones `read_*`. Hay una para cada necesidad pero las mas usadas son:

* `read_csv` para leer un archivo en formato csv.
* `read_excel` para leer un archivo en formato excel.

**Tarea**: Leer la documentacion de estas funciones.

### Operadores
Veamos que operaciones podemos aplicarle a una Serie de pandas

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame(index = list(range(10)), columns = ['a', 'b', 'c'])
df['a'] = np.random.rand(10,1)
df['b'] = 1
df['c'] = np.random.rand(10,1)
df

In [ ]:
# suma
df['a'] + df['b']

In [ ]:
# Podemos guardar la serie resultado en una nueva columna asi de facil:
df['d'] = df['a'] + df['b']
df

In [ ]:
# resta
df['a'] - df['b']

In [ ]:
# multiplicación
df['a'] * df['b']


## Ejercicio 2 (5min)
1. Utilizando el dataset de los pokemons, calcular para cada pokemon la suma de sus estadisticas (Puntos de salud + Ataque + Defensa + Ataque Especial + Defensa especial + Velocidad)

# El famoso NaN

Cuando trabajamos con un DataFrame, especialmente si es extenso, podemos tener problemas con lo valores NaN. Los NaN (“Not a Number“) son valores vacíos no computables que debemos tratar en nuestros conjuntos de datos antes de trabajar con ellos.

**Nota** Si bien NaN significa Not a Number, es un valor que no es un número, ni un string, ni ningun tipo de objeto conocido, ni siquiera `None`!. Es un valor especial que se utiliza en pandas para marcar aqullas posiciones de la tabla donde no hay ningún valor válido.

Veremos algunos códigos de Python que pueden ayudarnos a trabajar con los NaN.

* Podemos utilizar lo siguiente para saber si hay algún valor NaN en nuestro DataFrame

In [ ]:
data.isnull().values.any()


Hay valores faltantes! Para saber en que columnas se encuentran hacemos:

In [ ]:
data.isnull().any()


Vemos que tenemos valores faltantes en las columnas `neighbourhood_group` y otras. Para ver _cuantos_ valores faltan, podemos reemplazar `any` por `sum` en el codigo anterior:

In [ ]:
data.isnull().sum()

Podemos aplicar el método `sum` nuevamente para contar cuantos NaN tenemos en total:

In [ ]:
data.isnull().sum().sum()


Dependiendo que analisis querramos hacer, quizá tengamos que remover esos valores faltantes de nuestro DataFrame. Esto lo podemos hacer con el método `dropna`, veamos la ayuda para saber como utilizarlo:

In [ ]:
help(pd.DataFrame.dropna)

Aja! Lo podemos usar en dos modos. Para obtener un DataFrame sin los indices que tienen valores faltantes para alguna columna:

In [ ]:
data_sin_indices_con_valores_faltantes = data.dropna(axis='index')
print(data_sin_indices_con_valores_faltantes.shape)

In [ ]:
data_sin_indices_con_valores_faltantes

No nos queda ningun valor! Esto es porque la columna `neighbourhood_group` era vacia, no contenia ningun valor. Entonces en este caso puede ser mejor obtener el dataframe que tenga todos los datos - pero sin las columnas que tienen valores faltantes.

In [ ]:
data_sin_columnas_con_valores_faltantes = data.dropna(axis='columns')
print(data_sin_columnas_con_valores_faltantes.shape)

Otra opción, que puede ser útil en ciertos casos, es reemplazar todos los valores faltantes por algo que sea un valor típico para dicha variable. Para ejemplificar, veamos como podriamos "rellenar" los valores de `reviews_per_month` con 0, o con el valor promedio de todos los valores para los que si tengo el dato:

In [ ]:
data['reviews_per_month']

In [ ]:
data['reviews_per_month'].fillna(0)

In [ ]:
data['reviews_per_month'].fillna(data['reviews_per_month'].mean())

# Filtrado de datos

Muchas veces no queremos realizar el analisis sobre todo el dataFrame, si no solo sobre una porcion. Para ello pandas nos permite realizar filtrados.

El filtrado se hace por condiciones, es decir nos quedamos con el subconjunto de datos que cumple cierta restricción. Por ejemplo, para obtener todos los departamentos que tiene al menos dos reviews por mes:

In [ ]:
data[ data['reviews_per_month'] >= 2 ]

Se puede filtar por condiciones **compuestas**, es decir por proposiciones complejas que involucren operadores lógicos. Los operadores lógicos son los siguientes:

* Y: se deben cumplir ambas condiciones para que el dato este en el resultado, similar al `and` que hacemos en los condicionales de Python, pero acá se utiliza el operador ampersand &.
* O: se debe cumplir al menos una de las dos condiciones para que el datos este en el resultado, similar al `or` que hacemos en los condicionales de Python, pero acá se utiliza el operador pipe | .
* NO: no se debe cumplir la condicion para que el dato este en el resultado, similar al `not` nativo, solo que se utiliza la virgulilla como operador ~.

Por ejemplo, podemos obtener todos los departamentos que tengan al menos 2 reviews por mes y ademas al menos 90 reviews en total de la siguiente forma

In [ ]:
data[ (data['reviews_per_month'] >= 2) & (data['number_of_reviews'] >= 90) ]

**Nota**: Las condiciones para filtrar se pueden complejizar tanto como se desee. Es importante encerrar entre parentesis cada condicion para evitar errores.

Para filtrar por valores numericos, podemos utilizar los operadores de comparacion. Para filtrar por strings, tambien podemos filtrar por igualdad. Por ejemplo, podemos obtener todos los departamos que tengan `room_type` igual a "Private room":

In [ ]:
data[ data['room_type'] == "Private room" ]

Tambien podemos hacer busquedas mas complejas. Para verlo, carguemos la version del dataset de listings que contiene las descripciones de cada apartamento.

In [ ]:
data_completo = pd.read_csv('listings.csv.gz')
data_completo = data_completo.dropna(axis='index', subset=['description'])
data_completo.columns

In [ ]:
data_completo['description'].head(2)

¿Podremos encontrar todos los departamentos que mencionen la palabra "gato" (cat) en descripcion?

In [ ]:
data_completo[ data_completo['description'] == "cat" ]

In [ ]:
data_completo[ data_completo['description'].str.contains(" cat ") ]["description"].iloc[2]

El descriptor `str` nos da acceso a utilizar los métodos de strings sobre una serie que contiene strings. En el ejemplo utilizamos `contains` pero podriamos utilizar tambien split, replace, etc.





## Ejercicio 3 - hasta el fin de clase:

Descargar el archivo de distribución de empleo formal en el AMBA de este [link](https://www.datos.gob.ar/dataset/produccion-distribucion-empleo-formal-amba) (el que en su título dice hasta CLAE2)

1. Leer el archivo con pandas dataframe.
2. Determinar cual debe ser el indice del dataframe.
3. Realizar un analisis de valores faltantes, tomar acciones en consecuencia.
4.   Calcular la cantidad de departamentos y sectores (CLAE 2 dígitos) qué emplean:

  1.   solo hombres
  2.   solo mujeres
  3.   a ambos géneros

5. Calcular la remuneración media para hombres y para mujeres
6. Calcular la remuneración media para mujeres en el CLAE = 6

Nota: realizar consideración acerca del promedio de promedios

7. Leer la documentación, e investigar el uso, de los siguientes métodos de pandas:
  - where
  - mask
  - clip
  - sort_values
  - rename
  - unique